# Text Normalization

자주 마주치는 문제와 현실적인 해결 방법에 대해 알아보자.

## Extract Hanzi

주어진 텍스트에서 한자와 한자 아닌 것을 분리해야할 때가 있다. 한자의 유니코드 범위가 다양하기 때문에 이를 고려하여 추출한다. 

## Merge Duplicates & Variants

주어진 텍스트에서 다중코드자 및 이체자를  병합하면 분석 과정의 오류를 줄일 수 있다. 

In [43]:
# 한자 추출해 내기

import re

def get_cjk_unicode_range():
    cjk_unicode_range = {
        "Unified_Ideographs" : "\u4E00-\u9FCF",
        "Unified_Ideographs_ExA" : "\u3400-\u4DBF",
        "Unified_Ideographs_ExB" : "\u20000-\u2A6DF",
        "Unified_Ideographs_ExC" : "\u2A700-\u2B73F",
        "Unified_Ideographs_ExD" : "\u2B740-\u2B81F",
        "Compatibility_Ideographs" : "\uF900-\uFAFF",
        "Compatibility_Ideographs_Supplement": "\u2F800-\u2FA1F",
        "Radicals": "\u2F00-\u2FDF",
        "Radicals_Supplement": "\u2E80-\u2EFF",
        "Strokes": "\u31C0-\u31EF",
        "Ideographic_Description_Characters": "\u2FF0-\u2FFF"
    }

    cjk_range = "[" + "".join( list( cjk_unicode_range.values() ) ) + "]"
    none_cjk_range = "[^" + "".join( list( cjk_unicode_range.values() ) ) + "]"
    cjk_range_re = re.compile( cjk_range )
    none_cjk_range_re = re.compile( none_cjk_range )
#     cjk_range_re = re.compile( cjk_range , re.DEBUG)
#     none_cjk_range_re = re.compile( none_cjk_range , re.DEBUG)
    
    return ( cjk_range_re, none_cjk_range_re )

In [44]:
def get_dicts( dict_path ):
    with open( dict_path, 'r', encoding="utf-8") as fl:
        _dc = fl.readlines()
    dc = [ [ c.strip() for c in _d.split("\t") ] for _d in _dc ]
    return dc

def get_duplicates_dict():
    return get_dicts( "../data/hanzi_duplicates" )

def get_variants_dict():
    return get_dicts( "../data/hanzi_variants" )

In [45]:
class HanziText:
    
    def __init__(self):
        self.cjk_range_re, self.none_cjk_ragne_re = get_cjk_unicode_range()
        self.duplicates_dict = get_duplicates_dict()
        self.variants_dict = get_variants_dict()
        
    def extract_hanzi(  self, text ):
        return re.findall( self.cjk_range_re, text  )
        
    def exclude_hanzi(  self, text ):
        return re.findall( self.none_cjk_ragne_re, text  )
    
    def _merge_chr_pair(  self, dictionary, text ):
        _text = text + ""
        for a, b in dictionary:
            _text = _text.replace( a, b )
        return _text
    
    def merge_duplicates( self, text ):
        return self._merge_chr_pair( self.duplicates_dict,  text )
        
    def merge_variants(  self, text ):
        return self._merge_chr_pair( self.variants_dict,  text )

In [46]:
tn = HanziText()

In [47]:
text = "葛根四两　麻黄三两（去节）　桂二两（去皮）芍药二两（切）　甘草二两（炙）　生姜三两（切）大枣十二枚（掰）右七味（口父）咀，以水一斗，先煮麻黄葛根，减二升，去沫，内诸药，煮取三升，去滓，温服一升，复取微似汗，不须啜粥，余如桂枝法将息及禁忌。太阳与阳明合病者，必自下利，葛根汤主之。太阳与阳明合病，不下利，但呕者，葛根加半夏汤主之。"

print( "* 한자인 것: ", "".join( tn.extract_hanzi( text ) ) )
print( "* 한자 아닌 것:", "".join( tn.exclude_hanzi( text ) ) )

* 한자인 것:  葛根四两麻黄三两去节桂二两去皮芍药二两切甘草二两炙生姜三两切大枣十二枚掰右七味口父咀以水一斗先煮麻黄葛根减二升去沫内诸药煮取三升去滓温服一升复取微似汗不须啜粥余如桂枝法将息及禁忌太阳与阳明合病者必自下利葛根汤主之太阳与阳明合病不下利但呕者葛根加半夏汤主之
* 한자 아닌 것: 　（）　（）（）　（）　（）（）（），，，，，，，，，，，。，，。，，，。


In [48]:
hanzi_only = "".join( tn.extract_hanzi( text ) )
hanzi_dup_merged = tn.merge_duplicates( hanzi_only )
print(hanzi_dup_merged)

葛根四两麻黄三两去节桂二两去皮芍药二两切甘草二两炙生姜三两切大枣十二枚掰右七味口父咀以水一斗先煮麻黄葛根减二升去沫内诸药煮取三升去滓温服一升复取微似汗不须啜粥余如桂枝法将息及禁忌太阳与阳明合病者必自下利葛根汤主之太阳与阳明合病不下利但呕者葛根加半夏汤主之


In [49]:
hanzi_var_merged = tn.merge_variants( hanzi_dup_merged )
print( hanzi_var_merged )

葛根四兩麻黃三兩去節桂二兩去皮芍藥二兩切甘草二兩炙生姜三兩切大棗十二枚掰右七味口父咀以水一斗先煮麻黃葛根減二升去沫內諸藥煮取三升去滓溫服一升復取微似汗不須啜粥余如桂枝法將息及禁忌太陽與陽明合病者必自下利葛根湯主之太陽與陽明合病不下利但嘔者葛根加半夏湯主之


※ variants dictionary file를 이중코드자로 합쳐 놓치 않으면 병합되지 않는 글자가 발생할 수 있다. 

In [50]:
with open( "../data/hanzi_variants", 'r', encoding="utf-8" ) as fl:
    new_dict = tn.merge_duplicates( fl.read() )
    
with open( "../data/hanzi_variants", 'w', encoding="utf-8" ) as fl:
    fl.write( new_dict )